Notes (swahl 7/13):

Had to change urllib.request to urllib2.urlopen for python 2.7

Wikipedia searches use underscores between words, spaces don't work.

There is an obvious problem with names that will lead to dissambiguation pages (e.g. Drake -> Drake_(rapper) ). It would probably be easier to use a data source more specific to artists.


In [1]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib

In [2]:
#ArtistList = DataFrame(['drake_(rapper)', 'lady_gaga'],columns=['artist_name'])

#ArtistList = DataFrame.from_csv('../mode/mode-report-run-707064319e01.csv', sep='\t',index_col=False)
ArtistList = pd.DataFrame.from_csv('../billboard/billboard_artist_names_2010_15.csv')
DailyStats = pd.DataFrame()
ArtistList['name_length'] = 0
ArtistList.head()

,artist_name,name_length
0,Ke$ha,0
1,Eminem,0
2,Katy Perry,0
3,Rihanna,0
4,Bruno Mars,0


In [3]:
for i in range(len(ArtistList)):
    ArtistList['name_length'][i] = len(ArtistList['artist_name'][i])
#ArtistList = ArtistList[ArtistList['name_length']> 1]
#ArtistList = ArtistList[ArtistList['name_length'] <= 100]
#len(ArtistList)

/Users/swahl/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [4]:
ArtistList.head()

,artist_name,name_length
0,Ke$ha,5
1,Eminem,6
2,Katy Perry,10
3,Rihanna,7
4,Bruno Mars,10


In [5]:
import re
def cleanName(name):
    """
    Use regular expressions to change an artist name string into
    a valid wikipedia search string.
    """
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    r = re.compile(r"\s+")
    s = r.sub("_", s) # "a\nb\nc
    return s.lower()
  
cleanName('Black Keys, The')    

'the_black_keys'

In [6]:
ArtistList['artist_name_clean'] = ArtistList.artist_name.apply(cleanName)
ArtistList.artist_name_clean.head()

0         ke$ha
1        eminem
2    katy_perry
3       rihanna
4    bruno_mars
Name: artist_name_clean, dtype: object

In [12]:
fullList = ArtistList
#ArtistList = ArtistList.iloc[0:5]
ArtistList = fullList[44:]

In [8]:
# build up a list of month year tuples
year_months = [(str(y),"%02d" % m)  for y in range(2010,2015) for m in range(1,13)]
year_months += [(str(2015),"%02d" % m) for m in range(1,8)]
#year_months

In [14]:
reader = codecs.getreader("utf-8")
baseurl = "http://stats.grok.se/json/"
lang = 'en'
#year = '2015'
#month = '05'
DailyStats = DataFrame()
#for a in range(0, len(ArtistList) -1):
n = len(ArtistList)
for a in ArtistList.index.values:
#for a in ArtistList.index.values[41:100]:
    print a+1,'/',n
    for year,month in year_months:
        try:
            url= baseurl+lang+'/'+year+month+'/'
            #search_term = ArtistList['artist_name'][a]
            search_term = ArtistList['artist_name_clean'][a]
            url += search_term
            #print url
            #response = request.urlopen(url)
            response = urlopen(url)
            json_obj = load(reader(response))
            d = json_obj['daily_views']
            s  = pd.Series(d,index=d.keys())
            df = pd.DataFrame(s)
            df.columns = ['PageViews']
            df['Artist'] = ArtistList['artist_name'][a]
            df['search_term'] = search_term
            df['Language'] = lang
            DailyStats = DailyStats.append(df)

        except:
            print (ArtistList['artist_name'][a] )

45 / 587
46 / 587
47 / 587
48 / 587
49 / 587
50 / 587
51 / 587
52 / 587
53 / 587
54 / 587
55 / 587
56 / 587
57 / 587
58 / 587
59 / 587
60 / 587
61 / 587
62 / 587
63 / 587
64 / 587
65 / 587
66 / 587
67 / 587
68 / 587
69 / 587
70 / 587
71 / 587
72 / 587
73 / 587
74 / 587
75 / 587
76 / 587
77 / 587
78 / 587
79 / 587
80 / 587
81 / 587
82 / 587
83 / 587
84 / 587
85 / 587
86 / 587
87 / 587
88 / 587
89 / 587
90 / 587
91 / 587
92 / 587
93 / 587
94 / 587
95 / 587
96 / 587
97 / 587
98 / 587
99 / 587
100 / 587
101 / 587
102 / 587
103 / 587
104 / 587
105 / 587
106 / 587
107 / 587
108 / 587
109 / 587
110 / 587
111 / 587
112 / 587
113 / 587
114 / 587
115 / 587
116 / 587
117 / 587
118 / 587
119 / 587
120 / 587
121 / 587
122 / 587
123 / 587
124 / 587
125 / 587
126 / 587
127 / 587
128 / 587
129 / 587
130 / 587
131 / 587
132 / 587
133 / 587
134 / 587
135 / 587
136 / 587
137 / 587
138 / 587
139 / 587
140 / 587
141 / 587
142 / 587
143 / 587
144 / 587
145 / 587
146 / 587
147 / 587
148 / 587
149 / 587
150 /

In [15]:
#print url
print DailyStats.count()
DailyStats.tail()

PageViews      1209438
Artist         1209438
search_term    1209438
Language       1209438
dtype: int64


,PageViews,Artist,search_term,Language
2015-07-15,385,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-12,395,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-07,456,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-10,410,LunchMoney Lewis,lunchmoney_lewis,en
2015-07-11,326,LunchMoney Lewis,lunchmoney_lewis,en


In [16]:
#DailyStats[DailyStats.Artist == 'Eminem'].sort()
#DailyStats0_44 = DailyStats
#DailyStats45_end = DailyStats

In [18]:
#DailyStats_all = DailyStats0_44.append(DailyStats45_end)

In [19]:
# why are all of the entries 31?
#g = DailyStats.groupby('Artist')
g = DailyStats_all.groupby('Artist')


In [33]:
#DailyStats_all['date'] = DailyStats_all.index

In [32]:
#DailyStats_all

In [34]:
DailyStats_all = DailyStats_all.sort(['Artist','date'])

In [36]:
DailyStats_all = DailyStats_all[['date','Artist','search_term','Language','PageViews']]

In [37]:
DailyStats_all.to_csv('DailyWikiStats_from_billboard_2010_15.csv',index=False)